In [1]:
import numpy as np
import pandas as pd

allfiles = ['b_should_be_easy.in', 'c_no_hurry.in', 'd_metropolis.in', 'e_high_bonus.in']
alldfs = []

for file in allfiles:
    with open(file, 'r') as f:
        df = pd.DataFrame([ride.split(" ") for ride in f.readlines()],
                      dtype=float,
                      columns=['start_row', 'start_col', 'end_row', 'end_col', 'time_earliest', 'time_latest'])
        alldfs.append(df)

b_df, c_df, d_df, e_df = alldfs

"""
LET'S WORK WITH THE FIRST DATASET: b_should_be_easy.in
"""

# save useful variables
b_NUM_ROWS, b_NUM_COLS, b_NUM_VEHICLES, b_NUM_RIDES, b_NUM_BONUS, b_NUM_STEPS = b_df.loc[0, :]

# remove first row
b_df = b_df.loc[1:,:]

# compute distance
b_df['distance'] = np.abs(b_df.end_col - b_df.start_col) + np.abs(b_df.end_row - b_df.start_row)

# sort by earliest time and distance
b_df.sort_values(by=['time_earliest', 'distance'], inplace=True)

# add ride_id field
# b_df = b_df.reset_index(drop=True)
# b_df = b_df.reset_index().rename({'index': 'ride_id'}, axis=1)

b_df=b_df.astype(int)
print(b_df.head())


     start_row  start_col  end_row  end_col  time_earliest  time_latest  \
222        518        656      201      494            186          712   
217        666        188      504      995            208         2340   
168        502        138       42      160            228         1244   
1          395         43      296      955            244         3620   
235        662        421      314      445            281          656   

     distance  
222       479  
217       969  
168       482  
1        1011  
235       372  


/home/alvaro/.local/lib/python3.5/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alvaro/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
b_df.head()

start_row  start_col  end_row  end_col  time_earliest  time_latest  \
222        518        656      201      494            186          712   
217        666        188      504      995            208         2340   
168        502        138       42      160            228         1244   
1          395         43      296      955            244         3620   
235        662        421      314      445            281          656   

     distance  
222       479  
217       969  
168       482  
1        1011  
235       372

In [3]:
#df_vehicles lambda function to each ride yet to complete
def stepSubstract(row):
#     print('start sub',row)
    row['steps_available']=row['steps_available']-1
    row['steps_early']=row['steps_early']-1
    if row['steps_early']==0: #run ends
#         print(row)
        if isinstance(row['ride_id'],list):
            row['ride_id']=row['ride_id'].append(row['rideToComplete'])
        else:
            row['ride_id']=[row['ride_id']]
        row['rideToComplete']=-1
        if row['steps_available']==0:
            row['status']='stop'
        else: 
            row['status']='available'
#     print('end sub',row)
    return row
            
def assignRideToVehicle(dfVehicles,rideID,rideRow,dfRides):
    #get first vehicle capable
    vehicleAvailables=dfVehicles[(dfVehicles['status']=='available') &\
    ((dfVehicles['steps_available']==-1)|\
     (dfVehicles['steps_available']>= rideRow.distance))].vehicleID.values
    if len(vehicleAvailables)>0:
        for vID in vehicleAvailables:
    #assign it rideID
            if dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]==-1:
#                 print(rideRow)
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'rideToComplete']=rideID
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available']=rideRow['time_latest']
#                 print('in1',dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>0)
#                 print('in2',dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>rideRow['time_earliest'])
# mind it may still have steps_available to 0
            if (dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>0)&(dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_available'].values[0]>rideRow['time_earliest']):
#                 print('in3')            
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'rideToComplete']=dfVehicles.loc[dfVehicles['vehicleID']==vID,'rideToComplete'].apply(lambda x: x.append(rideID) if isinstance(x,list) else [rideID])
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'steps_early']=rideRow['time_earliest']
                dfVehicles.loc[dfVehicles['vehicleID']==vID,'status']='start'
            #         print(dfVehicles.head())
    #drop ride from ridesDF
#                 print('fanculo',dfVehicles.loc[dfVehicles['vehicleID']==vID])
    
#                 print('fanculo2',rideRow.head())
                dfRides=dfRides.drop(rideID)
#                 print('fanculo2',rideRow.head())
            
                return

In [4]:
df_vehicles=pd.DataFrame({'vehicleID':[],
                          'ride_id':[],
                          'steps_available':[],
                          'steps_early':[],
                          'status':[],
                          'rideToComplete':[]})
# rowVehicle='vehicleID':''.'rideID':[].'steps_available':'','status':'','rideToComplete':''

df_vehicles['vehicleID'] = range(int(b_NUM_VEHICLES))
df_vehicles['status'] = 'available'
df_vehicles['steps_available'] = -1
df_vehicles['steps_early'] = -1
df_vehicles['rideToComplete'] = -1

df_vehicles['ride_id'] = ''

In [5]:
b_df.head()

start_row  start_col  end_row  end_col  time_earliest  time_latest  \
222        518        656      201      494            186          712   
217        666        188      504      995            208         2340   
168        502        138       42      160            228         1244   
1          395         43      296      955            244         3620   
235        662        421      314      445            281          656   

     distance  
222       479  
217       969  
168       482  
1        1011  
235       372

In [6]:
for step in range(int(b_NUM_STEPS)):
    rides_to_assign = b_df[b_df.time_earliest == step]
    if len(rides_to_assign)>0:
#         print(rides_to_assign.head())
        for index, rowRide in rides_to_assign.iterrows():
            assignRideToVehicle(df_vehicles, index,rowRide, rides_to_assign)
    if(len(df_vehicles[df_vehicles['status']=='start'])>0):
        for index, rowV in df_vehicles[df_vehicles['status']=='start'].iterrows():
#             df_vehicles.ix[index,:]=stepSubstract(rowV)
            df_vehicles.loc[index,'steps_available']=rowV['steps_available']-1
            df_vehicles.loc[index,'steps_early']=rowV['steps_early']-1
            if df_vehicles.loc[index,'steps_early']==0: #run ends
        #         print(row)
                if df_vehicles.loc[index,'ride_id']=='':
                    df_vehicles.loc[index,'ride_id']=rowV['ride_id']
#                 if isinstance(rowV['ride_id'],list):
#                     df_vehicles.loc[index,'ride_id']=df_vehicles.loc[index,'ride_id'].append(rowV['rideToComplete'])
                else:
                    df_vehicles.loc[index,'ride_id']=df_vehicles.loc[index,'ride_id']+', '+rowV['ride_id']
                df_vehicles.loc[index,'rideToComplete']=-1
                if rowV['steps_available']==0:
                    df_vehicles.loc[index,'status']='stop'
                else: 
                    df_vehicles.loc[index,'status']='available'
#         break
#     break

In [7]:
df_vehicles

rideToComplete  ride_id     status  steps_available  steps_early  vehicleID
0              -1       -1  available               76            0          0
1              -1       -1  available              559            0          1
2              -1       -1  available              384            0          2
3              -1       -1  available             1204            0          3
4              -1       -1  available              375            0          4
5              -1       -1  available              121            0          5
6              -1       -1  available              835            0          6
7              -1       -1  available              795            0          7
8              -1       -1  available              101            0          8
9              -1       -1  available              956            0          9
10             -1       -1  available             1031            0         10
11             -1       -1  available              720            0         11
12             -1       -1  available              982            0         12
13             -1       -1  available             2576            0         13
14             -1       -1  available              295            0         14
15             -1       -1  available             2309            0         15
16             -1       -1  available              381            0         16
17             -1       -1  available             1106            0         17
18             -1       -1  available             2355            0         18
19             -1       -1  available             1363            0         19
20             -1       -1  available             1144            0         20
21             -1       -1  available              868            0         21
22             -1       -1  available              435            0         22
23             -1       -1  available              892            0         23
24             -1       -1  available             2010            0         24
25             -1       -1  available             2274            0         25
26             -1       -1  available              570            0         26
27             -1       -1  available             1847            0         27
28             -1       -1  available             2501            0         28
29             -1       -1  available             1581            0         29
..            ...      ...        ...              ...          ...        ...
70             -1       -1  available             3047            0         70
71             -1       -1  available              622            0         71
72             -1       -1  available             1334            0         72
73             -1       -1  available             1468            0         73
74             -1       -1  available             1138            0         74
75             -1       -1  available              733            0         75
76             -1       -1  available              561            0         76
77             -1       -1  available             3606            0         77
78             -1       -1  available              297            0         78
79             -1       -1  available              517            0         79
80             -1       -1  available             1363            0         80
81             -1       -1  available             1631            0         81
82             -1       -1  available             1621            0         82
83             -1       -1  available             1204            0         83
84             -1       -1  available             1657            0         84
85             -1       -1  available              627            0         85
86             -1       -1  available             1939            0         86
87             -1       -1  available             1135            0         87
88             -1       -1  available              56

In [8]:
b_df.shape

(300, 7)